In [ ]:
import numpy as np


def is_symmetric(arr: np.ndarray) -> bool:
    return np.array_equal(arr, arr[::-1])


def solve_ODE2_Dirichlet(a, b, c, s, x, u1=0, u2=0):
    import numpy as np
    from scipy.sparse import csr_matrix
    from scipy.sparse.linalg import spsolve
    # This function solves the 2nd ODE:
    # a[x]u''[x]+b[x]u'[x]+c[x]u[x]==s
    # with boundary conditions u[x[0]]=u0 and u[x[-1]]=u2
    # using central difference scheme

    # check a,b,c,s,x,u must be the same shape
    if not (a.shape == b.shape == c.shape == s.shape == x.shape):
        raise ValueError("a,b,c,s,x,u must be the same shape")

    # assemble the matrix A
    A2 = np.zeros((len(x), len(x)))
    B1 = np.zeros((len(x), len(x)))
    C0 = np.zeros((len(x), len(x)))
    for i in range(1, len(x) - 1):
        dx1 = x[i] - x[i - 1]
        dx2 = x[i + 1] - x[i]

        A2[i, i - 1] = 2 * dx2 / (dx1 * dx2 * (dx1 + dx2)) * a[i]
        A2[i, i] = -2 / (dx1 * dx2) * a[i]
        A2[i, i + 1] = 2 * dx1 / (dx1 * dx2 * (dx1 + dx2)) * a[i]

        B1[i, i - 1] = -(dx2 / dx1) / (dx1 + dx2) * b[i]
        B1[i, i] = (dx2**2 - dx1**2) / (dx1 * dx2 * (dx1 + dx2)) * b[i]
        B1[i, i + 1] = (dx1 / dx2) / (dx1 + dx2) * b[i]

        C0[i, i] = c[i]

    # assemble the right-hand side vector
    f = s.copy()

    # linear system with BC
    A = A2 + B1 + C0
    A[0, 0] = 1
    A[-1, -1] = 1
    f[0] = u1
    f[-1] = u2
    A_sparse = csr_matrix(A)

    res = spsolve(A_sparse, f)

    if (is_symmetric(a) and is_symmetric(b) and is_symmetric(c)
            and is_symmetric(s)):
        res = 0.5 * (res + np.flip(res))

    return res


def grad_y(u, y):
    import numpy as np
    N = np.size(u)
    dudy = np.zeros_like(u)
    for i in range(1, N - 1):
        dy1 = y[i] - y[i - 1]
        dy2 = y[i + 1] - y[i]
        dy12 = y[i + 1] - y[i - 1]
        dudy[i] = (-dy2 / dy1 * u[i - 1] + (dy2**2 - dy1**2) /
                   (dy1 * dy2) * u[i] + dy1 / dy2 * u[i + 1]) / dy12
    dudy[0] = (u[1] - u[0]) / (y[1] - y[0])
    dudy[-1] = (u[-1] - u[-2]) / (y[-1] - y[-2])
    return dudy


def grad_y2(u, y):
    import numpy as np
    N = np.size(u)
    dudy2 = np.zeros_like(u)
    for i in range(1, N - 1):
        dy1 = y[i] - y[i - 1]
        dy2 = y[i + 1] - y[i]
        dudy2[i] = (dy2 / (dy1 + dy2) * u[i - 1] - u[i] + dy1 /
                    (dy1 + dy2) * u[i + 1]) * 2 / (dy1 * dy2)
    dudy2[0] = dudy2[1]
    dudy2[-1] = dudy2[-2]
    return dudy2


def Cess_eddy_viscosity(Retau, y, nu, s=1.0):
    import numpy as np
    kappa = 0.426
    A = 25.4
    yplus = Retau * (1 - np.abs(y))
    f = kappa * Retau / 3 * (1 - y**2) * (1 + 2 * y**2) * (1 -
                                                           np.exp(-yplus / A))
    nut = 0.5 * np.sqrt(1 + s * np.power(f, 2)) - 0.5
    nut *= nu
    return nut


def solver_komega(nu_, Retau_, y_, k_0_, omega_0_, U_0_, max_iter_=500):
    # komega params
    beta = 3 / 40
    beta_star = 9 / 100
    sigma = 1 / 2
    sigma_star = 1 / 2
    gamma = 5 / 9

    utau = Retau_ * nu_
    omega_wall = 6 * nu_ / (beta * (y_[1] - y_[0])**2)

    error_U = []
    error_k = []
    error_omega = []
    k = k_0_
    omega = omega_0_
    U = U_0_
    for it in range(max_iter_):
        nut = k / omega
        dnutdy = grad_y(nut, y_)
        dUdy = -np.power(utau, 2) * y_ / (nu_ + nut)

        omega_new = solve_ODE2_Dirichlet(nu_ + sigma * nut, sigma * dnutdy,
                                         -beta * omega, -gamma * dUdy**2, y_,
                                         omega_wall, omega_wall)
        k_new = solve_ODE2_Dirichlet(nu_ + sigma_star * nut,
                                     sigma_star * dnutdy,
                                     -beta_star * omega_new, -nut * dUdy**2,
                                     y_)
        U_new = solve_ODE2_Dirichlet(nu_ + nut, dnutdy, np.zeros_like(y_),
                                     -np.ones_like(y_) * utau**2, y_)
        error_U.append(np.mean(np.abs(U - U_new)))
        error_k.append(np.mean(np.abs(k - k_new)))
        error_omega.append(np.mean(np.abs(omega - omega_new)))

        U = U_new
        k = k_new
        omega = omega_new
    return nut, k, omega, U, error_k, error_omega, error_U


def solver_kepsilon(nu_, Retau_, y_, k0_, eps0_, U0_, max_iter_=800):
    # k-epsilon params
    Cmu = 0.09
    sigma_eps = 1.3
    C1 = 1.44
    C2 = 1.92
    kappa = 0.41

    # solver
    Ny = np.size(y_)
    idx = slice(1, Ny - 1)
    utau = Retau_ * nu_

    error_U = []
    error_k = []
    error_eps = []
    alp = 0.2

    eps = eps0_
    k = k0_
    U = U0_
    for it in range(max_iter_):
        # solve nut
        nut = Cmu * k**2 / eps
        dnutdy = grad_y(nut, y_)
        dUdy = grad_y(U, y_)

        # solve k
        k_a = nut + nu_
        k_b = dnutdy
        k_c = np.zeros(Ny)
        k_c[idx] = -Cmu * k[idx] / nut[idx]
        k_s = -nut * dUdy**2
        k_new = solve_ODE2_Dirichlet(k_a, k_b, k_c, k_s, y_)
        error_k.append(np.mean(np.abs(k - k_new)))

        # solve eps
        eps_a = nu_ + nut / sigma_eps
        eps_b = dnutdy / sigma_eps
        eps_c = np.zeros(Ny)
        eps_s = np.zeros(Ny)
        eps_c[idx] = -C2 * eps[idx] / k[idx]
        eps_s[idx] = -C1 * nut[idx] * dUdy[idx]**2 * eps[idx] / k[idx]
        eps_w = 2 * nu_ * k[1] / np.power(y_[1] - y_[0], 2)
        eps_new = solve_ODE2_Dirichlet(eps_a, eps_b, eps_c, eps_s, y_, eps_w,
                                       eps_w)
        error_eps.append(np.mean(np.abs(eps - eps_new)))

        # solve velocity
        U_new = solve_ODE2_Dirichlet(nu_ + nut, dnutdy, np.zeros(Ny),
                                     -utau**2 * np.ones(Ny), y_)

        error_U.append(np.mean(np.abs(U - U_new)))
        # update - relaxation
        U = alp * U_new + (1 - alp) * U
        k = alp * k_new + (1 - alp) * k
        eps = alp * eps_new + (1 - alp) * eps

    return nut, k, eps, U, error_k, error_eps, error_U


##########################################################################################
##########################################################################################


def f_mu(y, utau_, nu_):
    yplus = (1 - np.abs(y)) * utau_ / nu_
    return 1 - np.exp(-0.0115 * yplus)


def f2(k, eps, nu):
    ReT = np.zeros_like(k)
    Ny = np.size(ReT)
    idx = slice(1, Ny - 1)
    ReT[idx] = k[idx]**2 / (eps[idx] * nu * 6)
    ReT[0] = ReT[1]
    ReT[-1] = ReT[-2]
    return 1 - 2 / 9 * np.exp(-ReT**2)


def solver_kepsilon_Chien(nu_,
                          Retau_,
                          y_,
                          k0_,
                          eps0_,
                          U0_,
                          nut0_,
                          max_iter_=800):
    # k-epsilon params
    Cmu = 0.09
    sigma_eps = 1.3
    C1 = 1.35
    C2 = 1.80

    # solver
    Ny = np.size(y_)
    idx = slice(1, Ny - 1)
    utau = Retau_ * nu_
    yplus = Retau_ * (1 - np.abs(y_))
    d = 1 - np.abs(y_)

    # solve iteration
    error_U = []
    error_eps = []
    error_k = []
    alp = 0.2
    k = k0_
    eps = eps0_
    U = U0_
    nut = nut0_
    for it in range(max_iter_):
        # solve nut
        dUdy = grad_y(U, y_)
        utau_ = np.sqrt(dUdy[0] * nu_)
        dplus = (1 - np.abs(y_)) * utau / nu_
        f_mu_val = f_mu(y_, utau_, nu_)
        dnutdy = grad_y(nut, y_)
        f2_val = f2(k, eps, nu_)

        # solve k
        k_a = nut + nu_
        k_b = dnutdy
        k_c = np.zeros(Ny)
        k_c[idx] = -Cmu * f_mu_val[idx] * k[idx] / nut[idx]
        k_c[idx] -= 2 * nu_ / d[idx]**2
        k_s = -nut * dUdy**2
        k_new = solve_ODE2_Dirichlet(k_a, k_b, k_c, k_s, y_)
        error_k.append(alp * np.mean(np.abs(k - k_new)))

        # solve eps
        eps_a = nu_ + nut / sigma_eps
        eps_b = dnutdy / sigma_eps
        eps_c = np.zeros(Ny)
        f2_val = f2(k, eps, nu_)
        eps_c[idx] = -2 * nu_ * np.exp(-0.5 * yplus[idx]) / d[idx]**2
        eps_c[idx] -= C2 * f2_val[idx] * eps[idx] / k[idx]
        eps_s = -C1 * Cmu * f_mu_val * k * dUdy**2
        eps_new = solve_ODE2_Dirichlet(eps_a, eps_b, eps_c, eps_s, y_)
        error_eps.append(alp * np.mean(np.abs(eps - eps_new)))

        # solve velocity
        U_new = solve_ODE2_Dirichlet(nu_ + nut, dnutdy, np.zeros(Ny),
                                     -utau**2 * np.ones(Ny), y_)

        error_U.append(alp * np.mean(np.abs(U - U_new)))
        # update - relaxation
        U = alp * U_new + (1 - alp) * U
        k = alp * k_new + (1 - alp) * k
        eps = alp * eps_new + (1 - alp) * eps
        nut[idx] = Cmu * f_mu_val[idx] * k[idx]**2 / eps[idx]

    return nut, k, eps, U, error_k, error_eps, error_U


######################################################################################
# S-A model functions


def f_t2(chi):
    import numpy as np
    c_t3 = 1.2
    c_t4 = 0.5
    return c_t3 * np.exp(-c_t4 * np.power(chi, 2))


def f_v1(chi):
    c_v1 = 7.1
    return np.power(chi, 3) / (np.power(chi, 3) + c_v1**3)


def f_v2(chi, f_v1_):
    return 1 - chi / (1 + chi * f_v1_)


def f_w(g):
    c_w3 = 2
    return g * np.power((1 + c_w3**6) / (g**6 + c_w3**6), 1 / 6)


def S_hat(Omega, nuh, d, f_v2_):
    Ny = np.size(d)
    kappa = 0.41
    res = np.zeros_like(d)
    res[1:Ny -
        1] = Omega[1:Ny - 1] + nuh[1:Ny - 1] * f_v2_[1:Ny - 1] / np.power(
            kappa * d[1:Ny - 1], 2)

    res[0] = res[1]
    res[-1] = res[-2]

    return res


def funcs_value(nuh, dUdy, y):
    d = np.abs(1 - np.abs(y))
    Ny = np.size(d)
    c_w2 = 0.3
    kappa = 0.41

    # chi
    chi = nuh / nu
    Omega = np.abs(dUdy)

    # f_v1
    f_v1_val = f_v1(chi)

    # f_v2
    f_v2_val = f_v2(chi, f_v1_val)

    # f_t2
    f_t2_val = f_t2(chi)

    # f_w
    S_hat_val = S_hat(Omega, nuh, d, f_v2_val)

    r_temp = np.zeros_like(d)
    r_temp[1:Ny - 1] = nuh[1:Ny - 1] / (S_hat_val[1:Ny - 1] * (kappa**2) *
                                        (d[1:Ny - 1]**2))
    r_temp[0] = r_temp[1]
    r_temp[-1] = r_temp[-2]
    r = np.minimum(r_temp, 10)
    g = r + c_w2 * (r**6 - r)
    f_w_val = f_w(g)

    return S_hat_val, f_t2_val, f_w_val


def solver_SA(nu_, Retau_, y_, U0_, nuh0_, max_iter_=500):
    # params
    sigma = 2 / 3
    c_b1 = 0.1355
    c_b2 = 0.622
    kappa = 0.41
    c_w1 = c_b1 / kappa**2 + (1 + c_b2) / sigma

    # mesh
    d = np.abs(1 - np.abs(y_))
    Ny = np.size(y_)

    # pressure gradient
    utau = Retau_ * nu_

    # run solver
    error_U = []
    error_nuh = []
    alp = 0.1
    U = U0_
    nuh = nuh0_
    nut = np.ones_like(y_) * nu_
    for it in range(max_iter_):
        dUdy = grad_y(U, y_)
        dnuhdy = grad_y(nuh, y_)
        # dnuhdy2 = grad_y2(nuh, y_)
        S_hat_val, f_t2_val, f_w_val = funcs_value(nuh, dUdy, y_)

        nuh_a = (nu_ + nuh) / sigma * np.ones_like(y_)
        nuh_b = np.zeros_like(y_)
        nuh_c = np.zeros_like(y_)
        nuh_s = np.zeros_like(y_)
        idx = slice(1, Ny - 1)
        nuh_s = -c_b1 * (1 - f_t2_val) * S_hat_val * nuh
        nuh_s[idx] += (c_w1 * f_w_val[idx] - c_b1 * f_t2_val[idx] /
                       (kappa**2)) * np.power(nuh[idx] / d[idx], 2)
        nuh_s -= (1 + c_b2) / sigma * np.power(dnuhdy, 2)
        # nuh_s -= nuh / sigma * dnuhdy2
        nuh_new = solve_ODE2_Dirichlet(nuh_a, nuh_b, nuh_c, nuh_s, y_)
        error_nuh.append(np.mean(np.abs(nuh - nuh_new)))

        dnutdy = grad_y(nut, y_)
        U_new = solve_ODE2_Dirichlet(nu_ + nut, dnutdy, np.zeros(Ny),
                                     -utau**2 * np.ones(Ny), y_)
        error_U.append(np.mean(np.abs(U - U_new)))

        # update
        U = alp * U_new + (1 - alp) * U
        nuh = alp * nuh_new + (1 - alp) * nuh
        nut = nuh * f_v1(nuh / nu_)

    return nuh, nut, U, error_nuh, error_U


In [ ]:
# Cess
import numpy as np
if __name__ == '__main__':

    # physics
    nu = 1 / 2800
    Retau = 180
    utau = Retau * nu

    # mesh
    Ny = 129
    y = np.cos(np.linspace(np.pi, 0, Ny))

    nut = Cess_eddy_viscosity(Retau, y, nu)
    dnutdy = grad_y(nut, y)
    U = solve_ODE2_Dirichlet(nu + nut, dnutdy, np.zeros(Ny),
                             -utau**2 * np.ones(Ny), y)

    np.savetxt(f'results-Cess-Retau-{Retau}.csv',
               np.column_stack((y, nut, U)),
               delimiter=',',
               header='y, nut, U')


In [ ]:
# standard k-omega
import matplotlib.pyplot as plt
import numpy as np

if __name__ == '__main__':

    # physics
    nu = 1 / 10150.0
    Retau = 550
    utau = Retau * nu

    # mesh
    Ny = 257
    y = np.cos(np.linspace(np.pi, 0, Ny))

    # initial condition and boundary condition
    k_0 = (1 - y**2) * 1.e-6
    omega_0 = np.ones_like(y) * 1.e-3
    U_0 = solve_ODE2_Dirichlet(nu * np.ones_like(y), np.zeros_like(y),
                               np.zeros_like(y), -np.ones_like(y) * utau**2, y)

    nut, k, omega, U, err_k, err_omega, err_U = solver_komega(nu,
                                                              Retau,
                                                              y,
                                                              k_0,
                                                              omega_0,
                                                              U_0,
                                                              max_iter_=700)

    np.savetxt(f'results-k-omega-Retau-{Retau}.csv',
               np.column_stack((y, k, omega, nut, U)),
               delimiter=',',
               header='y, k, omega, nut, U')

    fig, ax = plt.subplots(1, 5, figsize=(12, 3))
    ax[0].plot(y, k)
    ax[1].plot(y, omega)
    ax[2].plot(y, nut / (nu * Retau))
    ax[3].plot(Retau * (1 - np.abs(y)), U / utau, '.-')
    ax[4].plot(err_U, label='U')
    ax[4].plot(err_k, label='k')
    ax[4].plot(err_omega, label='omega')
    ax[0].set(xlim=(-1, 1), ylim=(0, 0.02))
    ax[2].set(xlim=(-1, 1), ylim=(0, 0.12))
    ax[3].set(xlim=(1.e-1, 800), ylim=(0, 25))
    ax[1].set_yscale('log')
    ax[3].set_xscale('log')
    ax[4].set_yscale('log')
    ax[4].legend()
    for i in range(5):
        ax[i].grid('on')

In [ ]:
# mixing-length
import matplotlib.pyplot as plt
import numpy as np

if __name__ == '__main__':
    # physics
    nu = 1 / 2800
    Retau = 180
    utau = Retau * nu

    kappa = 0.41

    # mesh
    Ny = 129
    y = np.cos(np.linspace(np.pi, 0, Ny))
    d = 1 - np.abs(y)

    # initial condition
    U = 1.5 * (1 - y**2)
    nut = np.zeros_like(y)

    #  mixing-length model - initial solution of U
    max_iter = 1000
    error = []
    for it in range(max_iter):
        dUdy = grad_y(U, y)
        nut = np.power(kappa * d, 2) * np.abs(dUdy)

        # solve U
        dnutdy = grad_y(nut, y)
        U_a = nu + nut
        U_b = dnutdy
        U_c = np.zeros_like(y)
        U_s = -np.ones_like(y) * utau**2
        U_new = solve_ODE2_Dirichlet(U_a, U_b, U_c, U_s, y)

        error.append(np.mean(np.abs(U - U_new)))
        U = U_new

    fig, ax = plt.subplots(1, 3, figsize=(9, 3))
    ax[0].plot(y, nut / (nu * Retau))
    ax[1].plot(Retau * (1 - np.abs(y)), U / utau, '.-')
    ax[2].plot(error)
    ax[1].set(xlim=(1.e-1, 200), ylim=(0, 25))
    ax[1].set_xscale('log')
    ax[2].set_yscale('log')
    for i in range(3):
        ax[i].grid('on')


In [ ]:
# standard k-epsilon
import matplotlib.pyplot as plt
import numpy as np

if __name__ == '__main__':
    # physics
    nu = 1 / 2800
    Retau = 180
    utau = Retau * nu

    # mesh
    Ny = 129
    idx = slice(1, Ny - 1)
    y = np.cos(np.linspace(np.pi, 0, Ny))

    # initial condition
    nut = Cess_eddy_viscosity(Retau, y, nu)
    dnutdy = grad_y(nut, y)
    U0 = solve_ODE2_Dirichlet(nu + nut, dnutdy, np.zeros(Ny),
                              -utau**2 * np.ones(Ny), y)
    k0 = np.ones(Ny) * 1.e-6
    eps0 = np.ones(Ny) * 1.e-9

    nut, k, eps, U, err_k, err_eps, err_U = solver_kepsilon(nu,
                                                            Retau,
                                                            y,
                                                            k0,
                                                            eps0,
                                                            U0,
                                                            max_iter_=600)

    np.savetxt(f'results-k-epsilon-Retau-{Retau}.csv',
               np.column_stack((y, k, eps, nut, U)),
               delimiter=',',
               header='y, k, epsilon, nut, U')

    fig, ax = plt.subplots(1, 5, figsize=(12, 2))
    ax[0].plot(y, k)
    ax[1].plot(y, eps)
    ax[2].plot(y, nut / (nu * Retau))
    ax[3].plot(Retau * (1 - np.abs(y)), U / utau)
    ax[4].plot(err_U, label='U')
    ax[4].plot(err_k, label='k')
    ax[4].plot(err_eps, label='eps')
    ax[4].legend()
    ax[0].set(xlim=(-1, 1), ylim=(0, 0.02))
    ax[2].set(xlim=(-1, 1), ylim=(0, 0.12))
    ax[3].set(xlim=(1.e-1, 200), ylim=(0, 20))
    ax[3].set_xscale('log')
    ax[4].set_yscale('log')
    for i in range(5):
        ax[i].grid('on')

In [ ]:
# Chien k-epsilon
import matplotlib.pyplot as plt
import numpy as np

if __name__ == '__main__':
    # physics
    nu = 1 / 10150
    Retau = 550
    utau = Retau * nu

    # params
    Cmu = 0.09
    sigma_eps = 1.3
    C1 = 1.44
    C2 = 1.92

    # mesh
    Ny = 257
    idx = slice(1, Ny - 1)
    y = np.cos(np.linspace(np.pi, 0, Ny))
    yplus = Retau * (1 - np.abs(y))
    d = 1 - np.abs(y)

    ################################################################################
    ################################################################################
    # initial condition
    nut = Cess_eddy_viscosity(Retau, y, nu)
    dnutdy = grad_y(nut, y)
    U = solve_ODE2_Dirichlet(nu + nut, dnutdy, np.zeros(Ny),
                             -utau**2 * np.ones(Ny), y)
    dUdy = grad_y(U, y)

    # fig, ax = plt.subplots(1, 2, figsize=(6, 3))
    # ax[0].plot(y, nut / (nu * Retau))
    # ax[1].plot(Retau * (1 - np.abs(y)), U / utau, '.-')
    # ax[1].set(xlim=(1.e-1, 200), ylim=(0, 25))
    # ax[1].set_xscale('log')
    # for i in range(2):
    #     ax[i].grid('on')

    # solve initial k, epsilon using U, nut from Cess model
    max_iter = 500
    error_k = []
    f_mu_val = f_mu(y, utau, nu)
    k = nut * np.abs(dUdy) / np.sqrt(Cmu)
    for it in range(max_iter):
        # solve k
        k_a = nut + nu
        k_b = dnutdy
        k_c = np.zeros(Ny)
        k_c[idx] = -Cmu * f_mu_val[idx] * k[idx] / nut[idx]
        k_c[idx] -= 2 * nu / d[idx]**2
        k_s = -nut * dUdy**2
        k_new = solve_ODE2_Dirichlet(k_a, k_b, k_c, k_s, y)
        error_k.append(np.mean(np.abs(k - k_new)))
        k = k_new

    # eps = Cmu * f_mu_val * k**2 / nut

    eps = np.zeros(Ny)
    # eps[idx] = Cmu * f_mu_val[idx] * k[idx]**2 / nut[idx]
    eps = C1 / C2 * nut * dUdy**2
    max_iter = 1200
    error_eps = []
    for it in range(max_iter):
        # solve eps
        eps_a = nu + nut / sigma_eps
        eps_b = dnutdy / sigma_eps
        eps_c = np.zeros(Ny)
        f2_val = f2(k, eps, nu)
        eps_c[idx] = -2 * nu * np.exp(-0.5 * yplus[idx]) / d[idx]**2
        eps_c[idx] -= C2 * f2_val[idx] * eps[idx] / k[idx]
        eps_s = -C1 * Cmu * f_mu_val * k * dUdy**2
        eps_new = solve_ODE2_Dirichlet(eps_a, eps_b, eps_c, eps_s, y)
        error_eps.append(np.mean(np.abs(eps - eps_new)))
        eps = eps_new
    # fig, ax = plt.subplots(1, 4, figsize=(12, 3))
    # ax[0].plot(y, k)
    # ax[1].plot(y, eps)
    # ax[2].plot(y, nut / (nu * Retau))
    # ax[3].plot(error_k)
    # ax[3].plot(error_eps)
    # ax[3].set_yscale('log')
    ################################################################################
    ################################################################################

    nut, k, eps, U, err_k, err_eps, err_U = solver_kepsilon_Chien(
        nu, Retau, y, k, eps, U, nut, max_iter_=600)

    np.savetxt(f'results-k-epsilon-Chien-Retau-{Retau}.csv',
               np.column_stack((y, k, eps, nut, U)),
               delimiter=',',
               header='y, k, epsilon, nut, U')

    fig, ax = plt.subplots(1, 5, figsize=(12, 2))
    ax[0].plot(y, k)
    ax[1].plot(y, eps)
    ax[2].plot(y, nut / (nu * Retau))
    ax[3].plot(Retau * (1 - np.abs(y)), U / utau)
    ax[4].plot(err_U[1:], label='U')
    ax[4].plot(err_k[1:], label='k')
    ax[4].plot(err_eps[1:], label='eps')

    # ax[0].set(xlim=(-1, 1), ylim=(0, 0.02))
    ax[2].set(xlim=(-1, 1))
    ax[3].set(xlim=(1.e-1, 200), ylim=(0, 25))
    ax[3].set_xscale('log')
    ax[4].set_yscale('log')
    ax[4].legend()
    for i in range(5):
        ax[i].grid('on')

In [ ]:
## SA model
import numpy as np
import matplotlib.pyplot as plt

if __name__ == '__main__':
    # physics
    nu = 1 / 10150.0
    Retau = 550
    utau = Retau * nu

    # mesh
    Ny = 257
    y = np.cos(np.linspace(np.pi, 0, Ny))

    # initial condition and boundary condition
    nuh0 = (1 - y**2) * nu * 1.e-6
    # nut = np.ones_like(y) * nu
    U0 = 1.5 * (1 - np.power(y, 2))

    # run solver
    nuh, nut, U, err_nuh, err_U = solver_SA(nu, Retau, y, U0, nuh0)
    np.savetxt(f'results-SA-Retau-{Retau}.csv',
               np.column_stack((y, nut, U)),
               delimiter=',',
               header='y, nut, U')

    fig, ax = plt.subplots(1, 4, figsize=(12, 3))
    ax[0].plot(y, nuh)
    ax[1].plot(y, nut / (nu * Retau))
    ax[2].plot(Retau * (1 - np.abs(y)), U / utau)
    ax[3].plot(err_U)
    ax[3].plot(err_nuh)
    ax[2].set(xlim=(1.e-1, 200))
    ax[2].set_xscale('log')
    ax[3].set_yscale('log')

In [ ]:
# compare models
import os
import sys
import numpy as np
import matplotlib.pyplot as plt

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from PyChannelResolvent import plot_config

plot_config.plot_config(latex=True)

if __name__ == '__main__':
    Retau = 180
    nu = 1 / 2800
    utau = Retau * nu
    dat_Cess = np.loadtxt(f'results-Cess-Retau-{Retau}.csv',
                          delimiter=',',
                          skiprows=1)
    dat_SA = np.loadtxt(f'results-SA-Retau-{Retau}.csv',
                        delimiter=',',
                        skiprows=1)
    dat_keps = np.loadtxt(f'results-k-epsilon-Retau-{Retau}.csv',
                          delimiter=',',
                          skiprows=1)
    dat_keps_Chien = np.loadtxt(f'results-k-epsilon-Chien-Retau-{Retau}.csv',
                                delimiter=',',
                                skiprows=1)
    dat_komega = np.loadtxt(f'results-k-omega-Retau-{Retau}.csv',
                            delimiter=',',
                            skiprows=1)

    y, nut_Cess, U_Cess = dat_Cess[:, 0], dat_Cess[:, 1], dat_Cess[:, 2]
    nut_SA, U_SA = dat_SA[:, 1], dat_SA[:, 2]
    nut_keps, U_keps = dat_keps[:, 3], dat_keps[:, 4]
    nut_keps_Chien, U_keps_Chien = dat_keps_Chien[:, 3], dat_keps_Chien[:, 4]
    nut_komega, U_komega = dat_komega[:, 3], dat_komega[:, 4]
    nut_dns = np.loadtxt(f'../Retau-{Retau}/RESULTS_EDDY_VISCOSITY/nut.dat',
                         skiprows=1)[:, 1]
    U_dns = np.loadtxt(f'../Retau-{Retau}/DATA-PLOT/mean-velocity.dat',
                       skiprows=1)[:, 2]
    yplus = Retau * (1 - np.abs(y))

    ##########################################################
    # plot
    sty_dns = {'linewidth': 0.75, 'color': 'k'}
    sty_SA = {'linewidth': 0.75, 'color': 'brown'}
    # sty_keps = {'linewidth': 0.75, 'color': 'brown'}
    sty_keps_Chien = {'linewidth': 0.75, 'color': 'orange'}
    sty_komega = {'linewidth': 0.75, 'color': 'dodgerblue'}
    sty_Cess = {'linewidth': 0.75, 'color': 'limegreen'}
    fig, ax = plt.subplots(1, 2, figsize=(5, 2))
    ax[0].plot(y, nut_dns / (nu * Retau), **sty_dns)
    ax[0].plot(y, nut_Cess / (nu * Retau), **sty_Cess)
    ax[0].plot(y, nut_SA / (nu * Retau), **sty_SA)
    # ax[0].plot(y, nut_keps / (nu * Retau), **sty_keps)
    ax[0].plot(y, nut_keps_Chien / (nu * Retau), **sty_keps_Chien)
    ax[0].plot(y, nut_komega / (nu * Retau), **sty_komega)

    ax[1].plot(yplus, U_dns / utau, **sty_dns, label='DNS')
    ax[1].plot(yplus, U_Cess / utau, **sty_Cess, label=r'Cess')
    ax[1].plot(yplus, U_SA / utau, **sty_SA, label=r'SA')
    # ax[1].plot(yplus,
    #            U_keps / utau,
    #            **sty_keps,
    #            label=r'standard $k$-$\varepsilon$')
    ax[1].plot(yplus,
               U_keps_Chien / utau,
               **sty_keps_Chien,
               label=r'$k$-$\varepsilon$')
    ax[1].plot(yplus, U_komega / utau, **sty_komega, label=r'$k$-$\omega$')

    ax[1].legend()

    ax[0].set(xlim=(-1, 1),
              ylim=(0, 0.12),
              yticks=np.linspace(0, 0.12, 7),
              xlabel=r'$y/h$',
              ylabel=r'$\nu_t/\nu Re_\tau^{-1}$')
    ax[1].set(xlim=(1.e-1, 800),
              ylim=(0, 20),
              yticks=np.linspace(0, 25, 6),
              xlabel=r'$y^+$',
              ylabel=r'$U^+$')
    ax[1].set_xscale('log')

    for i in range(2):
        ax[i].grid('on')

    plt.subplots_adjust(left=0.1,
                        right=0.95,
                        bottom=0.15,
                        top=0.95,
                        wspace=0.3)

    plt.savefig(f'RAS-channel-Retau-{Retau}.png')
    plt.savefig(f'RAS-channel-Retau-{Retau}.pdf')